# Generate new proteins using pre-trained genie

In [1]:
import os
import sys
import torch
import argparse
import numpy as np
from tqdm import tqdm, trange

np.set_printoptions(suppress=True)

gpu = input("GPU devices to use:")
if gpu == '':
    gpu = None
outdir = input("Save generated coordinates to:")
if outdir == '':
    outdir = 'outputs'
num_batches = input("number of batches:")
if num_batches == '':
    num_batches = 2
else:
    num_batches = int(num_batches)
batch_size = input("batch size:")
if batch_size == '':
    batch_size = 5
else:
    batch_size = int(batch_size)

if not os.path.exists(outdir):
    os.mkdir(outdir)

GPU devices to use: 0
Save generated coordinates to: 
number of batches: 1
batch size: 1


In [2]:
from genie.config import Config
from genie.diffusion.genie import Genie

config = Config('weights/configuration')
model = Genie.load_from_checkpoint('weights/genie_l_128_epoch=49999.ckpt', config=config)

device = 'cuda:{}'.format(gpu) if gpu is not None else 'cpu'
model = model.to(device)

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file weights/genie_l_128_epoch=49999.ckpt`


In [6]:
# sanity check
# min_length = 50
import analysis.utils as au
max_length = 128
max_n_res = model.config.io['max_n_res']
assert max_length <= max_n_res

# sample
# for length in trange(min_length, max_length + 1):
length = 128
for batch_idx in range(num_batches):
    mask = torch.cat([
        torch.ones((batch_size, length)),
        torch.zeros((batch_size, max_n_res - length))
    ], dim=1).to(device)
    ts = model.p_sample_loop(mask, verbose=False)[-1]
    for batch_sample_idx in range(ts.shape[0]):
        sample_idx = batch_idx * batch_size + batch_sample_idx
        coords = ts[batch_sample_idx].trans.detach().cpu().numpy()
        coords = coords[:length]
        np.savetxt(os.path.join(outdir, f'{length}_{sample_idx}.npy'), coords, fmt='%.3f', delimiter=',')

torch.Size([1, 128])


In [1]:
import numpy as np
outdir = 'outputs'
length = 128
sample_idx = 0
coords = np.loadtxt('outputs/128_0.npy', delimiter=',')

In [2]:
import analysis.utils as au
save_path = f'{outdir}/len_{length}_{sample_idx}.pdb'
au.write_prot_to_pdb(coords, save_path, no_indexing=True)

'outputs/len_128_0.pdb'

In [10]:
from plotly.subplots import make_subplots
import numpy as np
%matplotlib inline
#load coordinates
coords = np.loadtxt('outputs/128_0.npy', delimiter=',')
## Plot samples
num_res = np.sum(mask.cpu().numpy(), axis=-1)
nrows = int(np.sqrt(batch_size))
ncols = nrows
fig = make_subplots(
    rows=nrows, cols=ncols,
    specs=[[{'type': 'surface'}] * nrows]*ncols)
# Take last time step
# last_sample = coords
fig.update_layout(
    title_text=f'Samples',
    height=1000,
    width=1000,
)

sample_bb_3d = au.create_scatter(
    coords, mode='lines+markers', marker_size=3,
    opacity=1.0, name=f'Sample {1}: length={num_res[0]}')
fig.add_trace(sample_bb_3d, row=1, col=1)
        
fig.show(renderer='browser')

NameError: name 'mask' is not defined

self-consistency to PDMLM and alphafold

In [6]:
path_to_PDB="outputs/len_128_0.pdb"
output_dir="outputs/"

!python /Users/cplu/Downloads/Documents/ProteinMPNN/protein_mpnn_run.py \
        --pdb_path $path_to_PDB \
        --out_folder $output_dir \
        --num_seq_per_target 2 \
        --sampling_temp "0.1" \
        --seed 37 \
        --batch_size 1 \
        --ca_only

Using CA-ProteinMPNN!
----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
----------------------------------------
bias by residue dictionary is not loaded, or not provided
----------------------------------------
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
Generating sequences for: len_128_0
2 sequences of length 128 generated in 4.3915 seconds


In [49]:
import Bio
import Bio.PDB
import Bio.SeqRecord

def read_pdb(pdbcode, pdbfilenm):
    """
    Read a PDB structure from a file.
    :param pdbcode: A PDB ID string
    :param pdbfilenm: The PDB file
    :return: a Bio.PDB.Structure object or None if something went wrong
    """
    try:
        pdbparser = Bio.PDB.PDBParser(QUIET=True)   # suppress PDBConstructionWarning
        struct = pdbparser.get_structure(pdbcode, pdbfilenm)
        return struct
    except Exception as err:
        print(str(err), file=sys.stderr)
        return None 
def extract_seqrecords(pdbcode, struct):
    """
    Extracts the sequence records from a Bio.PDB structure.
    :param pdbcode: the PDB ID of the structure, needed to add a sequence ID to the result
    :param struct: a Bio.PDB.Structure object
    :return: a list of Bio.SeqRecord objects
    """
    ppb = Bio.PDB.PPBuilder()
    seqrecords = []
    for i, chain in enumerate(struct.get_chains()):
        # extract and store sequences as list of SeqRecord objects
        pps = ppb.build_peptides(chain)    # polypeptides
        print(struct.get_chains())
        seq = pps[0].get_sequence() # just take the first, hope there's no chain break
        seqid = pdbcode + chain.id
        seqrec = Bio.SeqRecord.SeqRecord(seq, id=seqid, 
            description="Sequence #{}, {}".format(i+1, seqid))
        seqrecords.append(seqrec)
    return seqrecords

def get_calphas_coord(struct):
    """
    Extracts the C-alpha atoms from a PDB structure.
    :param struct: A Bio.PDB.Structure object.
    :return: A list of Bio.PDB.Atom objects representing the C-alpha atoms in `struct`.
    """
    calphas = [ atom for atom in struct.get_atoms() if atom.get_fullname() == " CA " ]
    coords = np.array([atom.coord.tolist() for atom in calphas],dtype='float16')
    return coords

pdb1_struct = read_pdb('pdb1', pdb1_path)
pdb2_struct = read_pdb('pdb2', pdb2_path)

# only work for monomers
seq1 = str(extract_seqrecords('pdb1', pdb1_struct)[0].seq)
seq2 = str(extract_seqrecords('pdb2', pdb2_struct)[0].seq)

coords1 = get_calphas_coord(pdb1_struct)
coords2 = get_calphas_coord(pdb2_struct)

In [56]:
from tmtools.io import get_structure, get_residue_data
from tmtools.testing import get_pdb_path
def scTM(pdb1_path, pdb2_path):
    s = get_structure(pdb1_path)
    chain = next(s.get_chains())
    coords1, seq1 = get_residue_data(chain)
    
    s = get_structure(pdb2_path)
    chain = next(s.get_chains())
    coords2, seq2 = get_residue_data(chain)
    res = tm_align(coords1, coords2, seq1, seq2)
    return res.tm_norm_chain1

In [57]:
from tmtools import tm_align
pdb0_path = 'outputs/len_128_0.pdb'
pdb1_path = 'outputs/genie_128_1_a5a94/1_a5a94_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb'
pdb2_path = 'outputs/genie_128_2_3e40f/genie_128_2_3e40f_unrelaxed_rank_001_alphafold2_ptm_model_2_seed_000.pdb'
scTM(pdb0_path, pdb1_path)

0.4783427235910389

In [58]:
scTM(pdb0_path, pdb2_path)

0.37578811346275875

# Train dhaA dataset

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-g', '--gpus', type=str, help='GPU devices to use')
parser.add_argument('-c', '--config', type=str, help='Path for configuration file', required=True)
args = parser.parse_args()